# `swift_too` module

## Swift_PlanQuery example - querying planned *Swift* observations

### API version = 1.2, `swifttools` version = 2.4

#### Author: Jamie A. Kennea (Penn State)

The Swift_PlanQuery class allows for querying the database of observations have have been planned for observation by Swift, otherwise known as the "Pre-Planned Science Timeline" (PPST). This allows the user to find out not only what was planned, but what will be observed by Swift in the future.

Two important caveats:

1. For past observations, what should have been down is mostly an academic exercise, as due to TOO uploads and GRBs, Swift's pre-planned observations can be obliterated. If you want to know what Swift *did* observe, do a Swift_ObsQuery

2. Swift's planning is only done days into the future, so if you want to know what is going to happen weeks from now, you're out of luck.

This therefore is mostly useful if you want to know what Swift is observing *today* and *tomorrow*.

In [1]:
from swifttools.swift_too import PlanQuery
from datetime import datetime, timedelta

### First Query

Note: as of `swifttools 2.2`, anonymous login is the default, so if you do not give the `username` and `shared_secret`, it will log in as 'anonymous'

First query example: What is Swift planned to observe now? Quite easy to construct. Note as of `swifttools` 2.3, you can omit the `Swift_` from class names. Here is our first query:

In [2]:
plan = PlanQuery(begin=datetime.utcnow())
plan

2022-03-28 19:47:00,2022-03-28 19:57:00,ZTF21abcgnql,00014332012,600


So the above should have reported back a single target. When you just give `begin` with no `end`, it simply returns the observation which is taking place at the time given. So what is happening at this time tomorrow?

In [3]:
plan = PlanQuery(begin=datetime.utcnow() + timedelta(days=1))
plan

2022-03-29 19:48:00,2022-03-29 19:59:00,NGC 5236,00010978072,660


Did the above query return anything? If not, then there simply isn't a plan for tomorrow. So how can we determine how far into the future Swift has a plan for? The `ppstmax` parameter. Let's check it out:

In [4]:
f"PPSTs available up to {plan.ppstmax}"

'PPSTs available up to 2022-03-30 00:00:00'

This gives the date of the last planned target. If the query for what Swift is observing this time tomorrow didn't return anything, this date is probably the end of the UT day today. If the time you're running this is before midday on a Monday-Thursday, this is as expected. Typically the plan for tomorrow is submitted for upload around 12pm Eastern time. The exception to this rule is for the weekend plan. Typically on Thursday at 12pm we submit a plan for Friday and Saturday, and on Friday at 12pm we submit a plan for Sunday-Monday. This allows us to cover the weekend, while working a 5 day week.

### Second query example

Seeing a whole day's PPST. So let's see the whole day's Plan that goes farthest into the future.

In [5]:
dayplan = PlanQuery(begin=plan.ppstmax - timedelta(days=1), end=plan.ppstmax)
dayplan

2022-03-28 23:50:00,2022-03-29 00:00:00,OGP_2824,03104295003,600
2022-03-29 00:00:00,2022-03-29 00:07:00,OGP_3917,03103421003,420
2022-03-29 00:07:00,2022-03-29 00:25:00,saa-cold-218-0,00074100090,1080
2022-03-29 00:25:00,2022-03-29 00:35:00,MKN 6,00035461088,600
2022-03-29 00:35:00,2022-03-29 00:40:00,0716+714,00096041006,300
2022-03-29 00:40:00,2022-03-29 01:10:00,ZTF20abobpcb,00013649033,1800
2022-03-29 01:10:00,2022-03-29 01:20:00,1RXS J195500.6-160328,00083574001,600
2022-03-29 01:20:00,2022-03-29 01:23:19,SMC J0048.5-7256,00048753255,199
2022-03-29 01:23:19,2022-03-29 01:25:00,SMC J0113.8-7403,00048702206,101
2022-03-29 01:25:00,2022-03-29 01:30:00,N132D,00059114070,300
2022-03-29 01:30:00,2022-03-29 01:55:00,N132D,00059114071,1500


That's a lot of targets! Note that like Swift_ObsQuery it's showing you all the individual snapshots during the day. If you want to group these into observations, you can do that. Remember the `observations` method returns a dictionary, with obsid as the key, and the total observation details as the value. You can display it as a table in Jupyter Notebooks simply:

In [6]:
dayplan.observations

2022-03-28 23:50:00,2022-03-29 00:00:00,OGP_2824,03104295003,600,0
2022-03-29 00:00:00,2022-03-29 00:07:00,OGP_3917,03103421003,420,0
2022-03-29 00:07:00,2022-03-29 00:25:00,saa-cold-218-0,00074100090,1080,0
2022-03-29 00:25:00,2022-03-29 05:28:00,MKN 6,00035461088,2100,0
2022-03-29 00:35:00,2022-03-29 02:10:00,0716+714,00096041006,1200,0
2022-03-29 00:40:00,2022-03-29 08:55:00,ZTF20abobpcb,00013649033,2280,0
2022-03-29 01:10:00,2022-03-29 18:53:00,1RXS J195500.6-160328,00083574001,1980,0
2022-03-29 01:20:00,2022-03-29 01:23:19,SMC J0048.5-7256,00048753255,199,0
2022-03-29 01:23:19,2022-03-29 01:25:00,SMC J0113.8-7403,00048702206,101,0
2022-03-29 01:25:00,2022-03-29 14:24:00,N132D,00059114070,600,0
2022-03-29 01:30:00,2022-03-29 14:39:00,N132D,00059114071,2400,0


### A note about time formats

#### New features in `swifttools` 2.4

All times returned by `PlanQuery` come out in a time system that is derived from Swift's internal clock. The problem with this is that this clock, although close to UTC, is not actually UTC, due to the lack of handling of leap seconds, and also a slow drift in the clock itself. However, you can now correct this using the `clock_correct()` method. Let's do that:

In [7]:
query = PlanQuery(obsid='03102995005')
query.clock_correct()
query

2022-03-27 01:56:31.146818,2022-03-27 02:02:31.146793,OGP_2623,03102995005,359
2022-03-28 19:25:31.136722,2022-03-28 19:30:31.136702,OGP_2623,03102995005,299


So the table above now shows times that are specifically labelled as UTC. You'll note that the times now have fractions of a second, which is due to the clock correction that is applied. You can see the value of this clock correction (which includes both leap seconds and clock drift corrections), by looking at one of the date values, e.g.:

In [8]:
query[0].begin

670039020.0,2022-03-27 01:57:00,2022-03-27 01:56:31.146818,-28.853182


So for this observation, the UT Correction Factor (UTCF) is -24.012318s. For more detail on this correction, please take a look at the `Swift_Clock` and the notebook explaining that.